## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

# DBA Standalone Procedures

Sometimes you can't alway create the DBA procedures so you can run them as a one off script

> - Details of index statistics, seeks, scans etc (Query 1)
> - Details of missing index statistics, seeks, scans etc (Query 2)
> - Find blocking information (Query 3)
> - Find details of procedure stats (Query 4)
> - Find details of Server space (Query 5)
> - Find details of Index Columns (Query 6)
> - DBA Permissions  - Roles and permissions (Query 7)

> - Details of index statistics, seeks, scans etc (Query 1)

In [ ]:
/* DBA_IndexStats.sql  #SQL */
-- Updated to deal with read only secondary 
      
SELECT      
	@@Servername AS ServerName,
	DATABASEPROPERTYEX(DB_NAME(), 'Updateability') AS Updateability,
	db_name() AS DatabaseName, 
	c.name,
	o.name,      
	indexname=i.name,    
	i.type_desc, 
	i.index_id,      
	COALESCE(user_seeks,0) AS user_seeks ,  
	COALESCE(user_scans,0) AS user_scans,
	COALESCE(user_lookups,0) AS user_lookups,
	reads=COALESCE(user_seeks + user_scans + user_lookups,0),         
	writes =  COALESCE(user_updates,0),          
	rows = (SELECT SUM(p.rows) FROM sys.partitions p WHERE p.index_id = s.index_id AND s.object_id = p.object_id),      
	8 * (a.used_pages) AS 'Indexsize(KB)',  
	CASE      
		WHEN s.user_updates < 1 THEN 100      
		ELSE 1.00 * (s.user_seeks + s.user_scans + s.user_lookups) / s.user_updates      
	END AS reads_per_write,      
	'DROP INDEX ' + QUOTENAME(i.name)+' ON ' + QUOTENAME(c.name) + '.' + QUOTENAME(OBJECT_NAME(i.object_id)) as 'drop statement'      
FROM 
	sys.indexes i        
LEFT JOIN       
	sys.dm_db_index_usage_stats s ON i.index_id = s.index_id AND s.object_id = i.object_id         
LEFT JOIN       
	sys.objects o on i.object_id = o.object_id      
LEFT JOIN       
	sys.schemas c on o.schema_id = c.schema_id  
LEFT JOIN    
	 sys.partitions AS p ON p.OBJECT_ID = i.OBJECT_ID AND p.index_id = i.index_id  
LEFT JOIN   
	 sys.allocation_units AS a ON a.container_id = p.partition_id  
  
WHERE       
		(OBJECTPROPERTY(i.object_id,'IsUserTable') = 1  or OBJECTPROPERTY(i.object_id,'IsView') = 1)
	 --AND       
		--s.database_id = DB_ID() 
	 AND       
		i.name IS NOT NULL
	AND      
		i.is_unique_constraint = 0 

	AND user_seeks + user_scans + user_lookups = 0 AND i.type_desc  <> 'CLUSTERED'
	AND 			o.name IN  ('ms_transfers')
		
	--	OR  i.name IN ('Ixn_ms_returns_record_rr_initial_location_id_I','Ixn_ms_transfers_tr_group_tr_wave_id')
		
		--AND i.name IN (
		--'IX_ms_pallets_pal_despatch_pallet_id'
		--,'ix_ms_pallets_pal_warehouse_pal_open_user_pal_close_user_includes'
		----,'IDX_pal_po_primary_FILTERED'

	--	)

ORDER BY [Indexsize(KB)] desc








> - Details of missing index statistics, seeks, scans etc (Query 2)

In [ ]:
/* DBA_MissingIndexes.sql  #SQL */

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO


DECLARE
 @TableName sysname = null,
 @SchemeName sysname =null,
 @Sort Tinyint =1,
 @IncludeExcluded BIT = NULL,
 @CountOnly BIT = NULL
  
-----------------------------------------------------------------  
-- Object:   sp_DBA_MissingIndexes  
-- Written By:  
-- Date Written:   
-- Purpose:   Return missing index information
-- Usage:     
--    sp_DBA_MissingIndexes  @TableName='dbCodelookup'
-- Calls:   N/A  
-----------------------------------------------------------------  

SELECT
  migs.avg_total_user_cost * (migs.avg_user_impact / 100.0) * (migs.user_seeks + migs.user_scans) AS improvement_measure,
  mid.statement AS TableName, 
  so.name,
  mid.object_id,
  	 replace(replace('CREATE NONCLUSTERED INDEX Ixn_' + object_name(mid.object_id) +'_' 
			  +  case 
					when mid.equality_columns is not null 
					and mid.inequality_columns is not null 
					then replace(mid.equality_columns,', ','_') + '_' + replace(mid.inequality_columns,', ','_')
					when mid.equality_columns is not null 
					and mid.inequality_columns is null 
					then replace(mid.equality_columns,', ','_')
					when mid.inequality_columns is not null 
					then replace(mid.inequality_columns,', ','_')
				  end
			 +         case 
			when mid.included_columns is not null 
			then '_I'
			else ''
			end
		   + ' on ' 
		   + ss.name COLLATE DATABASE_DEFAULT + '.'
			+ object_name(mid.object_id) + ' (' + 
			case 
					when mid.equality_columns is not null 
					and mid.inequality_columns is not null 
					then mid.equality_columns + ',' + mid.inequality_columns
					when mid.equality_columns is not null 
					and mid.inequality_columns is null 
					then mid.equality_columns
					when mid.inequality_columns is not null 
					then mid.inequality_columns
			end
			+ ')' + char(10)
			+ 
			case 
			when mid.included_columns is not null 
			then 'Include (' + mid.included_columns + ')'
			else ''
			end,'[',''),']','') as CreateIndexStmt,
			 equality_columns AS equality_columns,
			 inequality_columns AS inequality_columns,
			 included_columns AS included_columns,
			migs.unique_compiles,
			migs.user_seeks, 
			migs.user_scans,
			migs.avg_total_user_cost, 
			migs.avg_user_impact,
			migs.last_user_seek, 
			migs.last_user_scan,
			migs.system_seeks, 
			migs.system_scans,
			migs.avg_total_system_cost, 
			migs.avg_system_impact,
			migs.last_system_seek, 
			migs.last_system_scan,
			(CONVERT(Numeric(19,6), migs.user_seeks)+CONVERT(Numeric(19,6), migs.unique_compiles))*CONVERT(Numeric(19,6), migs.avg_total_user_cost)*CONVERT(Numeric(19,6), migs.avg_user_impact/100.0) AS Score
FROM sys.dm_db_missing_index_groups mig
INNER JOIN sys.dm_db_missing_index_group_stats migs ON migs.group_handle = mig.index_group_handle
INNER JOIN sys.dm_db_missing_index_details mid ON mig.index_handle = mid.index_handle
LEFT JOIN sys.objects so on so.object_id=mid.object_id
LEFT JOIN sys.schemas ss ON ss.schema_id=so.schema_id
WHERE migs.avg_total_user_cost * (migs.avg_user_impact / 100.0) * (migs.user_seeks + migs.user_scans) > 10
AND so.name = COALESCE(@TableName,so.name) AND mid.database_id = db_id()
ORDER BY migs.avg_total_user_cost * migs.avg_user_impact * (migs.user_seeks + migs.user_scans) DESC









> - List blocking processes - Shows lead blocker and subsequent blocking chain (Query 3)

In [ ]:
/* DBA_BlockingChain.sql */
USE [master]
GO

SET ANSI_NULLS ON
GO

-----------------------------------------------------------------  
-- Object:   DBA_BlockingChain.sql  
-- Written By:  Martin Croft
-- Date Written:   
-- Purpose:   Shows lead blocking
-- Usage:     DBA_BlockingChain
-----------------------------------------------------------------  
-- Modified By  Modified Date  Description					Version 
-----------------------------------------------------------------  
-- {developer}  {date}    {description} {Version}  
-----------------------------------------------------------------  


IF (SELECT OBJECT_ID('TempDB.dbo.#T')) IS NOT NULL DROP TABLE #T
CREATE TABLE #T (spid INT, Blocked INT, BATCH VARCHAR(MAX))

INSERT INTO #T (Spid,blocked,Batch)
SELECT  spid ,
        blocked ,
        REPLACE(REPLACE(T.text, CHAR(10), ' '), CHAR(13), ' ') AS BATCH

FROM    sys.sysprocesses R
        CROSS APPLY sys.dm_exec_sql_text(R.sql_handle) T;

WITH    BLOCKERS ( SPID, BLOCKED, LEVEL, BATCH )
          AS ( SELECT   spid ,
                        blocked ,
                        CAST (REPLICATE('0', 4 - LEN(CAST (spid AS VARCHAR)))
                        + CAST (spid AS VARCHAR) AS VARCHAR(1000)) AS LEVEL ,
                        BATCH
               FROM     #T R
               WHERE    ( blocked = 0
                          OR blocked = spid
                        )
                        AND EXISTS ( SELECT *
                                     FROM   #T R2
                                     WHERE  R2.blocked = R.spid
                                            AND R2.blocked <> R2.spid )
               UNION ALL
               SELECT   R.spid ,
                        R.blocked ,
                        CAST (BLOCKERS.LEVEL
                        + RIGHT(CAST (( 1000 + R.spid ) AS VARCHAR(100)), 4) AS VARCHAR(1000)) AS LEVEL ,
                        R.BATCH
               FROM     #T AS R
                        INNER JOIN BLOCKERS ON R.blocked = BLOCKERS.SPID
               WHERE    R.blocked > 0
                        AND R.blocked <> R.spid
             )
    SELECT  N'    ' + REPLICATE(N'|         ', LEN(LEVEL) / 4 - 1)
            + CASE WHEN ( LEN(LEVEL) / 4 - 1 ) = 0 THEN 'HEAD -  '
                   ELSE '|------  '
              END + CAST (SPID AS NVARCHAR(10)) + N' ' + BATCH AS BLOCKING_TREE
    FROM    BLOCKERS
    ORDER BY LEVEL ASC;

GO




> - Find details of procedure stats (Query 4)

In [ ]:
/* DBA_ProcedureStats.sql  #SQL */
  
SELECT 
	 p.name AS [SP Name] ,  
	 SS.NAME,
	 qs.total_logical_reads AS [TotalLogicalReads] ,   
	 CASE WHEN qs.total_logical_reads  > 0 THEN	 qs.total_logical_reads / qs.execution_count ELSE 0
		  END	 AS [AvgLogicalReads] ,  
	 qs.execution_count ,   
	 CASE WHEN qs.total_elapsed_time > 0  THEN total_elapsed_time /1000 ELSE 0 END AS total_elapsed_timeMS,
	 
	 CASE WHEN qs.execution_count > 0  THEN (total_elapsed_time /1000)/qs.execution_count ELSE 0 END AS [avg_elapsed_timeMS] , 
	 	 CASE WHEN qs.execution_count > 0  THEN (total_elapsed_time /1000/1000)/qs.execution_count ELSE 0 END AS [avg_elapsed_timeSec] ,  
	 qs.cached_time,  
	 db_name() AS DbName, 
	 @@ServerName AS ServerName  
	 ,qs.plan_handle
FROM   
	sys.procedures AS p   
INNER JOIN   
	sys.dm_exec_procedure_stats AS qs ON p.[object_id] = qs.[object_id]  AND db_id() = database_id
LEFT JOIN 
	SYS.schemas SS ON SS.schema_id = p.schema_id

WHERE 1=1 
--AND p.name  LIKE '%%'
ORDER BY  8 desc



> - Server Space - Log/Data sizes with free space, summarised sever space, useful on dev/staging server to see where tou can get get space back (Query 5)

In [ ]:
/* DBA_DBSpace.sql  */


USE [master]
GO
-----------------------------------------------------------------  
-- Object:   sp_DBA_DBSpaceCalc  
-- Written By:  Martin Croft  
-- Date Written:   
-- Purpose:   Returns space information   
-- Template:  1.0   
-- Usage:     
--      
-- Calls:   N/A  
-----------------------------------------------------------------  
-- Modified By  Modified Date  Description  Version  
-----------------------------------------------------------------  
-- {developer}  {date}    {description} {Version}  
-- Added 2 decimal placing rounding 
-----------------------------------------------------------------  
SET NOCOUNT ON

IF (SELECT object_id('Tempdb..#LogFiles')) IS NOT NULL DROP TABLE #LogFiles
CREATE TABLE #LogFiles
(
 DBName VARCHAR(100)
,LogSize DECIMAL(12,6)
,LogSpaceuse DECIMAL(12,6)
,LogSpaceFree DECIMAL(12,6) DEFAULT (0)
)

INSERT INTO #LogFiles
EXEC ('DBCC sqlperf(logspace)')

UPDATE #logfiles
SET LogSpaceFree =(LogSize /100) * LogSpaceuse

IF (SELECT object_id('Tempdb..#FileStats')) IS NOT NULL DROP TABLE #FileStats
CREATE TABLE #FileStats 

(
	ID INT IDENTITY(1,1)
	,FileID INT
	,FileGroup INT
	,TotalExtents INT
	,UsedExtents INT
	,Name sysname
	,FileName VARCHAR(200)
	,DBNAME sysname NULL
	,Updated INT DEFAULT (0)
	,srvName VARCHAR(30)
)


IF (SELECT object_id('Tempdb..#Databases')) IS NOT NULL DROP TABLE #Databases
CREATE TABLE #Databases
(
	 ID INT IDENTITY(1,1)
	,DBID INT
	,Name VARCHAR(200)
)

 --Get all database that are online and not snapshots
INSERT INTO #Databases (DBID,Name)
SELECT 
	database_id,
	Name 
FROM 
	master.sys.databases
WHERE 
		state_desc='ONLINE' 
	AND 
		source_database_id IS NULL


DECLARE @MaxID INT, @Loop INT

DECLARE @ExecStr VARCHAR(200), @Name VARCHAR(100),@Rowcount INT

SELECT @MaxID =MAX(ID) FROM #Databases
SET @Loop=1

WHILE @Loop <= @MaxID 
BEGIN
	SELECT @Name = Name from #databases where ID =@loop
	SELECT @ExecStr='USE ['+@Name+']; DBCC showfilestats' 

--Insert file stats into temp table for each DB

	INSERT INTO #FileStats 
	(	FileID
		,FileGroup
		,TotalExtents
		,UsedExtents
		,Name
		,FileName
	)
	
	EXEC (@ExecStr)
	SET @rowcount =@@rowcount 
	

	--SELECT @Loop , (@loop+@rowcount)

	UPDATE #FileStats
	SET DBNAME = @Name
		,Updated =1
		,srvName =@@servername
	WHERE 
		updated =0

	SET @loop=@loop+1
END

--Database Sizes 

SELECT 
	 @@servername AS SrvName
	,DBName
	,fileid
	,DATABASEPROPERTYEX( DBName, 'RECOVERY' ) AS Model
	,ROUND(cast(((TotalExtents * 64 *1.0 )/1024) as decimal(10,2)),-2) TotalSizeMg
	,cast((UsedExtents * 64 *1.0) /1024 as decimal (10,2)) UsedSizeMg
	,cast(ROUND(((TotalExtents * 64 *1.0 )/1024 )-((UsedExtents * 64 *1.0) /1024 ),2,-2)as decimal(10,2)) FreeMg
FROM 
	#filestats
ORDER BY 
	DBNAME,fileid
	---ROUND(((TotalExtents * 64 *1.0 )/1024 )-((UsedExtents * 64 *1.0) /1024 ),2,-2)  DESC

 --Data than can be reclaimed back
SELECT 
	SrvName
	,ROUND(CAST(SUM(ROUND(((TotalExtents * 64 *1.0 )/1024 )-((UsedExtents * 64 *1.0) /1024 ),2,-2))AS VARCHAR(50)),-2) [FreeMg - Data Size that can be claimed back]
FROM 
	#filestats
GROUP BY
	SrvName

 --Data than can be reclaimed back
SELECT 
	SrvName
	,SUM(cast((UsedExtents * 64 *1.0) /1024 as decimal (10,2))) [TotalUsedSizeMg]
FROM 
	#filestats
GROUP BY
	SrvName
 
--Dabtabase log sizes
SELECT
	 DBName
	,ROUND(CAST(LogSize AS VARCHAR(50)),-2) [LogSize Mg]
	,ROUND(CAST(LogSpaceuse AS VARCHAR(50)),-2)[LogSpaceused %]
	,ROUND(CAST((LogSize-LogSpaceFree) AS VARCHAR(50)),-2)[LogSpaceFree Mg]
FROM 
	#logfiles
ORDER BY 
	LogSpaceFree DESC

--Log space that can be reclaimed  

SELECT 
	ROUND(CAST(SUM(LogSize-LogSpaceFree)AS VARCHAR(MAX)),2) [Log Space that can be reclaimed] 
FROM 
	#logfiles

> - Index Columns - Show the index columns in the index and any included columns (Query 6)

In [ ]:
/* Get index columns , also show included columns */

DECLARE 
	@tableName sysname = NULL

  SET NOCOUNT ON    
-----------------------------------------------------------------        
-- Object:     sp_DBA_GetIndexColumns      
-- Written By:         
-- Date Written:  19/10/2010       
-- Usage:           
-- sp_DBA_GetIndexColumns 
-- sp_DBA_GetIndexColumns @tableName ='tbl_Scheme'
-----------------------------------------------------------------        
-- {developer}  {date}    {description} {Version}        
-----------------------------------------------------------------        
--https://www.ptr.co.uk/blog/sql-server-display-indexes-their-columns-included-columns
DECLARE @TempTable AS TABLE (SchemaName VARCHAR(100), 
							 ObjectID INT, 
							 TableName VARCHAR(100), 
							 IndexID INT, 
							 IndexName VARCHAR(MAX), 
							 ColumnID INT, 
							 column_index_id INT, 
							 ColumnNames  VARCHAR(500), 
							 IncludeColumns  VARCHAR(MAX), 
							 NumberOfColumns INT, 
							 IndexType  VARCHAR(20),
							 LastColRecord INT);

WITH CTE_Indexes (SchemaName, ObjectID, TableName, IndexID, IndexName, ColumnID, column_index_id, ColumnNames, IncludeColumns, NumberOfColumns, IndexType)
AS
(
SELECT s.name, t.object_id, t.name, i.index_id, i.name, c.column_id, ic.index_column_id,
		CASE ic.is_included_column WHEN 0 THEN CAST(c.name AS VARCHAR(5000)) ELSE '' END, 
		CASE ic.is_included_column WHEN 1 THEN CAST(c.name AS VARCHAR(5000)) ELSE '' END, 1, i.type_desc
	FROM  sys.schemas AS s
		JOIN sys.tables AS t ON s.schema_id = t.schema_id
			JOIN sys.indexes AS i ON i.object_id = t.object_id
				JOIN sys.index_columns AS ic ON ic.index_id = i.index_id AND ic.object_id = i.object_id
					JOIN sys.columns AS c ON c.column_id = ic.column_id AND c.object_id = ic.object_id
						AND ic.index_column_id = 1
UNION ALL
SELECT s.name, t.object_id, t.name, i.index_id, i.name, c.column_id, ic.index_column_id,
		CASE ic.is_included_column WHEN 0 THEN CAST(cte.ColumnNames + ', ' + c.name AS VARCHAR(5000))  ELSE cte.ColumnNames END, 
		CASE  
			WHEN ic.is_included_column = 1 AND cte.IncludeColumns != '' THEN CAST(cte.IncludeColumns + ', ' + c.name AS VARCHAR(5000))
			WHEN ic.is_included_column =1 AND cte.IncludeColumns = '' THEN CAST(c.name AS VARCHAR(5000)) 
			ELSE '' 
		END,
		cte.NumberOfColumns + 1, i.type_desc
	FROM  sys.schemas AS s
		JOIN sys.tables AS t ON s.schema_id = t.schema_id
			JOIN sys.indexes AS i ON i.object_id = t.object_id
				JOIN sys.index_columns AS ic ON ic.index_id = i.index_id AND ic.object_id = i.object_id
					JOIN sys.columns AS c ON c.column_id = ic.column_id AND c.object_id = ic.object_id 
					JOIN CTE_Indexes cte ON cte.Column_index_ID + 1 = ic.index_column_id  
					--JOIN CTE_Indexes cte ON cte.ColumnID + 1 = ic.index_column_id  
							AND cte.IndexID = i.index_id AND cte.ObjectID = ic.object_id

)
INSERT INTO  @TempTable 
SELECT *, RANK() OVER (PARTITION BY ObjectID, IndexID ORDER BY NumberOfColumns DESC) AS LastRecord FROM CTE_Indexes AS cte;

SELECT SchemaName, TableName, IndexName, ColumnNames, IncludeColumns, IndexType FROM @TempTable
WHERE LastColRecord = 1
AND TableName = COALESCE(@TableName,TableName)
ORDER BY objectid, TableName, ColumnNames, IndexName





> - DBA Permissions  - Roles and permissions (Query 7)

In [8]:
/* DBA_Permissions.sql  #SQL */
  
-----------------------------------------------------------------  
-- Object:   sp_DBA_DBPermissions  
-- Written By:  
-- Date Written:   
-- Purpose:   Returns permissions for all databases   
-- Template:  1.0   
-- Usage:     
--    sp_DBA_DBPermissions  
-- Calls:   N/A  
-----------------------------------------------------------------  
-- Modified By  Modified Date  Description					Version 
-- MAC			16/11/2010	   Modified to use table var	2.0
-----------------------------------------------------------------  
-- {developer}  {date}    {description} {Version}  
-----------------------------------------------------------------  

DECLARE  @SecurityDetails TABLE
(ServerName sysname,DatabaseName sysname,member_principal_name sysname,principal_type_desc sysname,role_name sysname)

 INSERT INTO @SecurityDetails
	   
EXEC sp_MSForEachDB  N'USE [?]; 
			
	   
	   SELECT @@ServerName,
	   DB_name(),
	   rm.member_principal_name, 
	   rm.principal_type_desc, 
	   rm.role_name
FROM    sys.database_principals p
right outer JOIN (
				  select role_principal_id, 
				  dp.type_desc as principal_type_desc, 
				  member_principal_id,user_name(member_principal_id) as member_principal_name,
				  user_name(role_principal_id) as role_name

				  from sys.database_role_members rm
				  INNER JOIN sys.database_principals dp
					ON    rm.member_principal_id = dp.principal_id
				 
				   ) rm
ON   rm.role_principal_id = p.principal_id

Where principal_type_desc in (''Sql_User'',''WINDOWS_USER'',''WINDOWS_GROUP'')'

SELECT * FROM @SecurityDetails

GO




Commands completed successfully.

Total execution time: 00:00:00.261

ServerName,DatabaseName,member_principal_name,principal_type_desc,role_name
JURGEN,master,dbo,SQL_USER,db_owner
JURGEN,tempdb,dbo,SQL_USER,db_owner
JURGEN,model,dbo,SQL_USER,db_owner
JURGEN,msdb,dbo,SQL_USER,db_owner
JURGEN,msdb,MS_DataCollectorInternalUser,SQL_USER,SQLAgentUserRole
JURGEN,msdb,MS_DataCollectorInternalUser,SQL_USER,db_ssisoperator
JURGEN,msdb,MS_DataCollectorInternalUser,SQL_USER,dc_admin
JURGEN,msdb,##MS_PolicyEventProcessingLogin##,SQL_USER,PolicyAdministratorRole
JURGEN,msdb,##MS_PolicyTsqlExecutionLogin##,SQL_USER,PolicyAdministratorRole
JURGEN,MIH,dbo,SQL_USER,db_owner


## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)